
## 1. **Exploración y Comprensión de la Estructura de Datos**

### a. Categorías y Atributos
- **Obtener el árbol de categorías:**  
  Utilizaremos el endpoint `/sites/MLA/categories` para recuperar todas las categorías del sitio (por ejemplo, Mercado Libre Argentina). Esto nos ayudará a entender en qué contextos se publican los ítems y a segmentarlos según su naturaleza.
  
- **Detalle de una categoría:**  
  Con endpoints como `/categories/{id}` (por ejemplo, `/categories/MLA1744` para “Autos y Camionetas”), podemos conocer la estructura jerárquica, los atributos obligatorios y opcionales, y la configuración específica de cada categoría. Esto es clave para saber qué campos (títulos, descripciones, imágenes, atributos técnicos, etc.) tenemos disponibles para comparar los productos.

- **Atributos específicos y valores top:**  
  Usaremos también `/categories/{id}/attributes` para conocer los atributos específicos de una categoría. Además, el endpoint de **top values** nos ayudará a identificar cuáles son los valores más comunes para ciertos atributos (por ejemplo, las marcas más usadas), lo que puede servir para validar o enriquecer nuestro análisis.

### b. Predictor de Categorías
- Con el recurso `/domain_discovery/search` podremos, a partir de un título de producto, predecir la categoría a la que pertenece. Esto es útil si queremos validar que estamos comparando ítems dentro de la misma categoría o si necesitamos filtrar productos que efectivamente son similares.

---

## 2. **Extracción de Datos: Cómo Recopilar la Información**

### a. Extracción Masiva mediante Búsqueda
- **Uso del endpoint de búsqueda:**  
  Emplearemos la llamada a `/sites/MLA/search` con parámetros `category`, `q`, `limit` y `offset` para extraer un conjunto representativo de publicaciones. Por ejemplo:
  ```bash
  curl -X GET "https://api.mercadolibre.com/sites/MLA/search?category=MLA1000&offset=0&limit=50"
  ```
  Esto nos permitirá recopilar títulos, imágenes y otros datos relevantes de cada ítem.

### b. Paginación y Manejo de Resultados
- **Limit y Offset:**  
  La documentación explica cómo utilizar los parámetros `limit` y `offset` para obtener bloques de resultados. Esto es esencial para recolectar una cantidad suficiente de datos sin sobrecargar el sistema y para realizar un muestreo representativo de los productos.

---



## Obtener las Categorías del Sitio

### Mercado Libre Colombia

Este script conecta con la API de Mercado Libre para obtener las categorías disponibles en el sitio de Colombia (MCO).

---

### Proceso

1. **Conexión a la API:**  
   Se realiza una solicitud GET al endpoint:  
   `https://api.mercadolibre.com/sites/MCO/categories`.

2. **Validación de la Respuesta:**  
   - Si la solicitud es exitosa (`status_code = 200`), se procesa el listado de categorías.  
   - Si falla, se imprime un mensaje de error con el código de estado.

3. **Procesamiento:**  
   Se imprime el número de categorías obtenidas y su contenido en formato JSON.

---

### Posibles Errores

- **Error en la solicitud:** Problemas de conexión o URL incorrecta.  
- **Respuesta vacía:** Problemas temporales de la API o falta de datos en el sitio especificado.


In [1]:
import requests
import json


# Definimos la URL del endpoint de categorías para MLA
CATEGORY_URL = "https://api.mercadolibre.com/sites/MCO/categories"


def get_categories():
    """Obtiene la lista de categorías desde la API de MercadoLibre."""
    response = requests.get(CATEGORY_URL)

    if response.status_code == 200:
        categories = response.json()
        print("Número de categorías obtenidas:", len(categories))
        print(json.dumps(categories, indent=4, ensure_ascii=False))
    else:
        print("Error en la solicitud:", response.status_code)


if __name__ == "__main__":
    get_categories()


Número de categorías obtenidas: 32
[
    {
        "id": "MCO1747",
        "name": "Accesorios para Vehículos"
    },
    {
        "id": "MCO441917",
        "name": "Agro"
    },
    {
        "id": "MCO1403",
        "name": "Alimentos y Bebidas"
    },
    {
        "id": "MCO1071",
        "name": "Animales y Mascotas"
    },
    {
        "id": "MCO1367",
        "name": "Antigüedades y Colecciones"
    },
    {
        "id": "MCO1368",
        "name": "Arte, Papelería y Mercería"
    },
    {
        "id": "MCO1384",
        "name": "Bebés"
    },
    {
        "id": "MCO1246",
        "name": "Belleza y Cuidado Personal"
    },
    {
        "id": "MCO40433",
        "name": "Boletas para Espectáculos"
    },
    {
        "id": "MCO1039",
        "name": "Cámaras y Accesorios"
    },
    {
        "id": "MCO1743",
        "name": "Carros, Motos y Otros"
    },
    {
        "id": "MCO1051",
        "name": "Celulares y Teléfonos"
    },
    {
        "id": "MCO1648",
        

## Explorar el Detalle de una Categoría Específica

###
 {
        "id": "MCO1000",
        "name": "Electrónica, Audio y Video"
 }

Este script obtiene detalles específicos de una categoría en el marketplace de Mercado Libre usando su API.

---

## Proceso

1. **Definición de la Categoría:**  
   Se especifica el `ID` de la categoría a explorar, por ejemplo, `"MCO1000"`.

2. **Conexión a la API:**  
   Se realiza una solicitud GET al endpoint:  
   `https://api.mercadolibre.com/categories/{categoria_id}`.  
   Aquí, `{categoria_id}` es reemplazado por el ID de la categoría.

3. **Validación de la Respuesta:**  
   - Si la solicitud es exitosa (`status_code = 200`), se procesa el JSON con los detalles de la categoría.  
   - Si falla, se muestra un mensaje de error con el código de estado.

4. **Visualización:**  
   Se imprime información clave de la categoría, como:
   - Nombre.
   - Atributos obligatorios.
   - Subcategorías.
   - Estructura jerárquica.

---

## Posibles Errores

- **Error en la solicitud:** Ocurre si el ID de categoría es incorrecto o hay problemas de conexión.
- **Falta de información:** La categoría puede no tener datos relevantes si es un nodo vacío o no está bien configurada en el marketplace.


In [2]:
import requests
import json


# Definimos la URL base de la API de MercadoLibre
BASE_URL = "https://api.mercadolibre.com/categories"


def get_category_details(category_id):
    """Obtiene detalles de una categoría específica desde la API de MercadoLibre."""
    url = f"{BASE_URL}/{category_id}"
    response = requests.get(url)

    if response.status_code == 200:
        category_details = response.json()
        print("Detalle de la categoría:")
        print(json.dumps(category_details, indent=4, ensure_ascii=False))
    else:
        print("Error en la solicitud de detalle de categoría:", response.status_code)


if __name__ == "__main__":
    CATEGORY_ID = "MCO1000"
    get_category_details(CATEGORY_ID)


Detalle de la categoría:
{
    "id": "MCO1000",
    "name": "Electrónica, Audio y Video",
    "picture": "https://http2.mlstatic.com/storage/categories-api/images/943ec641-717e-49cb-8a34-2f40ba367f5a.png",
    "permalink": "https://www.mercadolibre.com.co/c/electronica-audio-y-video",
    "total_items_in_this_category": 1118751,
    "path_from_root": [
        {
            "id": "MCO1000",
            "name": "Electrónica, Audio y Video"
        }
    ],
    "children_categories": [
        {
            "id": "MCO3690",
            "name": "Accesorios Audio y Video",
            "total_items_in_this_category": 69026
        },
        {
            "id": "MCO431414",
            "name": "Accesorios para TV",
            "total_items_in_this_category": 38198
        },
        {
            "id": "MCO3835",
            "name": "Audio",
            "total_items_in_this_category": 426937
        },
        {
            "id": "MCO5054",
            "name": "Cables",
            "total_i

# Realizar una Búsqueda de Ítems en una Categoría

"Electrónica, Audio y Video"

Este script extrae hasta 1000 productos de una categoría específica utilizando la API de Mercado Libre.

---

## Proceso

1. **Definir Categoría:**
   - Se especifica el `CATEGORY_ID` de la categoría a analizar. Por ejemplo:
     - `"MCO1000"`: Electrónica, Audio y Video.

2. **Conexión a la API:**
   - Se realiza una solicitud GET al endpoint:  
     `https://api.mercadolibre.com/sites/MCO/search?category={CATEGORY_ID}&offset={offset}`.

3. **Paginación:**
   - La API devuelve resultados en bloques de 50 productos.
   - El script recorre las páginas hasta alcanzar un máximo de 1000 productos.

4. **Almacenamiento:**
   - Los productos extraídos se almacenan en una lista y luego se convierten a un DataFrame de Pandas.

5. **Visualización:**
   - Se muestran las primeras filas del DataFrame con `df.head()`.

---

## Salida

El script genera un DataFrame que contiene información de los productos, como `id`, `title`, `price`, y otros campos proporcionados por la API.


In [3]:
import requests
import pandas as pd
import time
from tqdm import tqdm


# Configuración de parámetros
CATEGORY_ID = "MCO1000"  # ID de la categoría a analizar
MAX_PRODUCTS = 1000  # Número máximo de productos a extraer
OFFSET_STEP = 50  # Tamaño del paginado de la API
API_BASE_URL = "https://api.mercadolibre.com/sites/MCO/search"


def fetch_products(category_id, max_products=1000, offset_step=50):
    """
    Extrae productos de la categoría especificada desde la API de MercadoLibre.

    :param category_id: ID de la categoría a consultar.
    :param max_products: Número máximo de productos a obtener.
    :param offset_step: Cantidad de productos por solicitud (límite de la API: 50).
    :return: Lista de productos extraídos.
    """
    products = []

    for offset in tqdm(range(0, max_products, offset_step), desc="Extrayendo productos"):
        url = f"{API_BASE_URL}?category={category_id}&offset={offset}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            products.extend(data.get("results", []))  # Agrega productos si existen
        else:
            print(f"Error en la consulta de offset {offset}: {response.status_code}")

        time.sleep(0.5)  # Pequeña pausa para no sobrecargar la API

    return products


def main():
    """Función principal que ejecuta la extracción de productos y los muestra en un DataFrame."""
    products = fetch_products(CATEGORY_ID, MAX_PRODUCTS, OFFSET_STEP)

    if products:
        df = pd.json_normalize(products)  # Aplana estructuras anidadas
        print(df.head())  # Muestra las primeras filas
    else:
        print("No se obtuvieron productos.")


if __name__ == "__main__":
    main()


Extrayendo productos: 100%|██████████| 20/20 [00:34<00:00,  1.72s/it]

              id                                              title condition  \
0  MCO2739654108  Audífonos Inalámbricos Bluetooth Sleve Pulse A...       new   
1  MCO2653035526  Televisor Kalley 40 Pulgadas K-rtv40hd Smart T...       new   
2  MCO1505219611  Bocina Sony Portátil Inalámbrica Ult Field 1 |...       new   
3  MCO2738127712               Televisor Samsung Smart 65 Qled Q70d       new   
4  MCO2769358618  Amazon Echo Dot 5th Gen Con Asistente Virtual ...       new   

                   thumbnail_id catalog_product_id listing_type_id  \
0  693995-MLU73095898992_112023        MCO28584565    gold_special   
1  798711-MLU79285460287_092024        MCO40340550    gold_special   
2  880581-MLA79774965373_102024        MCO41604757    gold_special   
3  864505-MLA79253232452_092024        MCO38778158    gold_special   
4  976672-MLU72754995740_112023        MCO19795010        gold_pro   

  sanitized_title                                          permalink  \
0                  h

# Elección de una Categoría Específica para el Challenge

## Introducción

Para abordar el challenge de identificar y agrupar productos similares en el marketplace, se ha decidido centrar el análisis en una única categoría. Esta estrategia permite desarrollar un prototipo robusto y profundo, además de facilitar la validación de los resultados.

## Motivación y Justificación de la Elección

La decisión de enfocarse en la categoría **"Electrónica, Audio y Video" (ID: MCO1000)** se fundamenta en diversas razones, tanto metodológicas como personales, desde la perspectiva del participante:

- **Afinidad por la Tecnología:**  
  El candidato muestra un especial interés en el ámbito tecnológico, y la categoría de Electrónica, Audio y Video abarca una amplia gama de productos innovadores. Esto resulta motivador para aplicar técnicas avanzadas de procesamiento de lenguaje natural y análisis de imágenes. Así mismo, por una experiencia personal al usar la aplicación de Mercado Libre para poner en venta unos transistores y sensores de temperatura. 

- **Calidad y Abundancia de Datos:**  
  Esta categoría ofrece una gran cantidad de publicaciones con información detallada (títulos, descripciones, imágenes, etc.), lo que facilita la extracción y el preprocesamiento de datos, permitiendo la construcción de modelos más precisos para comparar y agrupar ítems.

- **Homogeneidad en los Atributos:**  
  Trabajar en una sola categoría simplifica el análisis, ya que los productos comparten características y atributos similares. Esto reduce la complejidad en el preprocesamiento y mejora la consistencia en el análisis de similitud.

- **Enfoque y Profundidad del Análisis:**  
  Centrarse en una única categoría permite profundizar en el estudio de sus características particulares y optimizar los algoritmos de similitud y clustering, evitando la dispersión que implicaría abordar múltiples dominios simultáneamente.
